In [1]:
import tensorflow as tf
import numpy as np
import os


In [2]:
model = tf.keras.models.load_model("models/efficientnet_fruit.h5")
model.summary()




Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda_1 (Lambda)           (None, 224, 224, 3)       0         
                                                                 
 efficientnetb0 (Functional  (None, 7, 7, 1280)        4049571   
 )                                                               
                                                                 
 global_average_pooling2d_2  (None, 1280)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dropout_2 (Dropout)         (None, 1280)              0         
                                                                 
 dense_2 (Dense)             (None, 6)                 7686      
                                                                 
Total params: 4057257 (15.48 MB)
Trainable params: 7

In [6]:
from tensorflow.keras.utils import image_dataset_from_directory

img_size = (224, 224)
batch_size = 32

data_dir = "original_data_set/test"   # or train directory

dataset = image_dataset_from_directory(
    data_dir,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=False
)


Found 2698 files belonging to 6 classes.


# create feature extractor model 

In [9]:
feature_extractor = tf.keras.Model(
    inputs=model.input,
    outputs=model.layers[-3].output   # GAP layer output
)

feature_extractor.summary()


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda_1_input (InputLayer  [(None, 224, 224, 3)]     0         
 )                                                               
                                                                 
 lambda_1 (Lambda)           (None, 224, 224, 3)       0         
                                                                 
 efficientnetb0 (Functional  (None, 7, 7, 1280)        4049571   
 )                                                               
                                                                 
 global_average_pooling2d_2  (None, 1280)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
Total params: 4049571 (15.45 MB)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 4049571 (15.45 MB)
___________________

In [10]:
X_features = []
y_labels = []

for images, labels in dataset:
    features = feature_extractor.predict(images)
    X_features.append(features)
    y_labels.append(labels.numpy())


1/1 [==============================] - 1s 1s/step


# Convert to NumPy Arrays (IMPORTANT)

In [11]:
X_features = np.vstack(X_features)
y_labels = np.hstack(y_labels)

print("Feature matrix shape:", X_features.shape)
print("Labels shape:", y_labels.shape)


Feature matrix shape: (2698, 1280)
Labels shape: (2698,)


# save feature matrix

In [15]:
np.save("X_features.npy", X_features)
np.save("y_labels.npy", y_labels)

print("Feature matrix saved!")

Feature matrix saved!


# Print first 10 features of first image

In [12]:
print(X_features[0][:10])


[-0.1602852  -0.10288349 -0.10776821 -0.19370365 -0.16084635 -0.21411557
 -0.20097567 -0.18416272 -0.17599094 -0.09426355]


In [22]:
print(y_labels[2000])

4
